In [142]:
from transformers import AutoTokenizer
import pandas as pd
import csv
import numpy as np
from glob import glob
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

In [143]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")

In [144]:
sent = "This is, a real; sentence. Not really Legal...\nBut it is\nok."
tok_sent = tokenizer(sent)
print(tok_sent)
tokens = tokenizer.convert_ids_to_tokens(tok_sent["input_ids"])
print(tokens)

{'input_ids': [101, 1188, 1110, 117, 170, 1842, 132, 5650, 119, 1753, 1541, 10800, 119, 119, 119, 1252, 1122, 1110, 21534, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'This', 'is', ',', 'a', 'real', ';', 'sentence', '.', 'Not', 'really', 'Legal', '.', '.', '.', 'But', 'it', 'is', 'ok', '.', '[SEP]']


In [145]:
files = glob("./documents/lines/train/*.txt")
dataset_train = []
for file in files: 
    text = open(file, "r")
    original = text.readlines()
    for i in range(len(original)):
        original[i]= original[i].strip('\n')
        dataset_train += [original[i]]

In [146]:
files = glob("./documents/lines/dev/*.txt")
dataset_dev = []
for file in files: 
    text = open(file, "r")
    original = text.readlines()
    for i in range(len(original)):
        original[i]= original[i].strip('\n')
        dataset_dev += [original[i]]

In [147]:
df = pd.read_csv("./dataset_v20230110.tsv", delimiter="	", quoting=csv.QUOTE_NONE, encoding='utf-8')
df.head()

,document,content,tag
0,ROSENBLATT v. BAER_MCL,MR. JUSTICE BRENNAN delivered the opinion of t...,Announcing function
1,ROSENBLATT v. BAER_MCL,A jury in New Hampshire Superior Court awarded...,Describing the adjudicated facts
2,ROSENBLATT v. BAER_MCL,Respondent alleged that the column contained d...,Describing procedural events
3,ROSENBLATT v. BAER_MCL,In the interval between the trial and the deci...,Describing the adjudicated facts
4,ROSENBLATT v. BAER_MCL,We there held that consistent with the First a...,Recalling a SCOTUS decision


In [148]:
sentences = []
ner_tags = []
fin_tags = []
tokenizeds_train = []

for i in range(len(dataset_train)):
    tok_sent = tokenizer(dataset_train[i])
    #print(type(tok_sent))
    tokens = tokenizer.convert_ids_to_tokens(tok_sent["input_ids"])
    sentences += tokens[1:-1]
    n_tag = np.zeros(len(sentences), dtype=int)
    n_tag[-1] = 1
    ner_tags += n_tag.tolist()
    tok_sent["labels"] = n_tag
    #tokenizeds_dev+= [tok_sent]

for i in range(0, len(sentences), (510)):
    tokens = ['CLS'] + sentences[i:i + 510] + ['SEP']
    fin_tags = [0] + ner_tags[i:i + 510] + [0]
    tokeniz = tokenizer(tokens, is_split_into_words=True)
    tokeniz["labels"] = fin_tags
    tokenizeds_train += [tokeniz]
print(tokenizeds_train)
data_train = {"labels": fin_tags, "tokens": tokenizeds_train}

[{'input_ids': [101, 140, 15928, 115, 1853, 147, 108, 108, 1646, 108, 108, 157, 2240, 108, 108, 9855, 26660, 108, 108, 142, 108, 108, 162, 108, 108, 23580, 4653, 1103, 4893, 1104, 1103, 2031, 119, 2577, 5788, 170, 2877, 192, 108, 108, 187, 2875, 1104, 5871, 108, 108, 1129, 108, 108, 1112, 26661, 117, 170, 1352, 7396, 1538, 16287, 1907, 1352, 1231, 108, 108, 1143, 1181, 108, 108, 178, 1279, 117, 1743, 158, 119, 156, 119, 140, 119, 204, 15684, 108, 108, 125, 113, 171, 114, 113, 122, 114, 117, 8267, 2368, 1103, 1426, 1103, 107, 169, 107, 3767, 1106, 2789, 1852, 1105, 5663, 107, 6351, 13957, 1104, 1157, 5419, 112, 2877, 2266, 119, 112, 107, 6809, 191, 119, 1985, 117, 4062, 108, 108, 124, 158, 119, 156, 119, 3164, 108, 108, 125, 117, 21033, 113, 1876, 114, 113, 1679, 16408, 108, 108, 187, 1465, 108, 108, 182, 114, 113, 186, 108, 108, 190, 1186, 108, 108, 14086, 1403, 21902, 108, 108, 3621, 191, 119, 6121, 117, 27993, 158, 119, 156, 119, 14162, 117, 18783, 113, 2507, 114, 114, 119, 1706, 219

In [159]:
sentences = []
ner_tags = []
fin_tags = []
tokenizeds_dev = []

for i in range(len(dataset_dev)):
    tok_sent = tokenizer(dataset_dev[i])
    #print(type(tok_sent))
    tokens = tokenizer.convert_ids_to_tokens(tok_sent["input_ids"])
    sentences += tokens[1:-1]
    n_tag = np.zeros(len(sentences), dtype=int)
    n_tag[-1] = 1
    ner_tags += n_tag.tolist()
    tok_sent["labels"] = n_tag
    #tokenizeds_dev+= [tok_sent]

for i in range(0, len(sentences), (510)):
    tokens = ['CLS'] + sentences[i:i + 510] + ['SEP']
    fin_tags = [0] + ner_tags[i:i + 510] + [0]
    tokeniz = tokenizer(tokens, is_split_into_words=True)
    tokeniz["labels"] = fin_tags
    print(len(tokeniz['input_ids']))
    tokenizeds_dev += [tokeniz]
print(tokenizeds_dev)
data_dev = {"labels": fin_tags, "tokens": tokenizeds_dev}

676
679
695
635
591
740
624
683
668
636
603
636
619
625
650
680
698
650
625
583
594
665
631
571
571
560
108
[{'input_ids': [101, 140, 15928, 147, 108, 108, 1646, 108, 108, 157, 2240, 108, 108, 9855, 148, 108, 108, 142, 2249, 108, 108, 26546, 108, 108, 141, 108, 108, 162, 4653, 1103, 4893, 1104, 1103, 2031, 119, 1109, 12979, 10060, 117, 13036, 1106, 1103, 1311, 1118, 1103, 2538, 1104, 1103, 27711, 108, 108, 24438, 10060, 117, 2790, 1115, 1103, 170, 108, 108, 172, 118, 16408, 108, 108, 14516, 1181, 4103, 1138, 1103, 5052, 1104, 12654, 1107, 1155, 4771, 12369, 108, 108, 188, 119, 1109, 1268, 1106, 12654, 1110, 1103, 1268, 1106, 3903, 5052, 1104, 12654, 119, 3969, 1457, 108, 108, 187, 5345, 108, 108, 1657, 191, 119, 1994, 117, 3993, 108, 108, 127, 158, 119, 156, 119, 5046, 108, 108, 129, 117, 5599, 108, 108, 127, 113, 2219, 114, 119, 1188, 1692, 20251, 1107, 1103, 5618, 1104, 2694, 22830, 5052, 1115, 1521, 1106, 1103, 11033, 108, 108, 174, 1104, 170, 12369, 2906, 1104, 170, 16150, 19445, 1

In [150]:
label_list = ["0", "eos"]

In [151]:
id2label = {
    0: "0",
    1: "eos"
}

label2id = {
    "0": 0,
    "eos": 1
}

In [152]:
model = AutoModelForTokenClassification.from_pretrained(
        "distilbert/distilbert-base-uncased", 
        num_labels=2, 
        id2label=id2label, 
        label2id=label2id)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [153]:
import evaluate

seqeval = evaluate.load("seqeval")

In [154]:
def compute_metrics(p):

    predictions, labels = p

    predictions = np.argmax(predictions, axis=2)

    true_predictions = [

        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]

        for prediction, label in zip(predictions, labels)

    ]

    true_labels = [

        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]

        for prediction, label in zip(predictions, labels)

    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)

    return {

        "precision": results["overall_precision"],

        "recall": results["overall_recall"],

        "f1": results["overall_f1"],

        "accuracy": results["overall_accuracy"],

    }

In [156]:
training_args = TrainingArguments(
    output_dir="./models/distilbert-v0",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenizeds_train,
    eval_dataset=tokenizeds_dev,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 0/14 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: The size of tensor a (766) must match the size of tensor b (512) at non-singleton dimension 1